In [1]:
from collections import deque
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import codecs
import re
from webdriver_manager.chrome import ChromeDriverManager
import time

In [ ]:
def get_page_elements(driver,url):
    driver.get(url)
    # Extract all URLs and navigation data
    anchor_elements = driver.find_elements(By.TAG_NAME, 'a')
    all_elements = [{"type":"button","element":element,"next":None} for element in anchor_elements]
    input_elements = driver.find_elements(By.CSS_SELECTOR, "input[type='search']")
    all_elements.extend([{"type":"input","element":element,"next":None} for element in input_elements])
    return all_elements

In [ ]:
def get_page(tabs,url,driver): 
    if url != None and "https://www.ip-label.fr/" in url:
        url = url.split('#')[0]
        exist = False
        for page in tabs:
            if page["page"] == url or page["page"]+"#" == url:
                exist = True
        if exist:
            return page["index"]
        else:
            # Execute JavaScript to open a new tab.
            script = f"window.open('{url}', '_blank');"
            driver.execute_script(script)
            index=len(tabs)
            tabs.append({"page":url,"index":index,"page_elements" : []})
            return index
    else:
        return None
        
    

def open_tab(url,tabs,element,driver):
        index = get_page(tabs,url,driver)
        if index != None:
            # Switch to the new tab.
            new_tab_handle = driver.window_handles[index]
            driver.switch_to.window(new_tab_handle)
            element["next"] = url
                

In [ ]:
def input_xpath(element):
    tag_name = element.tag_name
    input_class = element.get_attribute("class")
    input_id = element.get_attribute("id")
    input_name = element.get_attribute("name")
    aria_label = element.get_attribute("aria-label")
    xpath = f"//{tag_name}[@class='{input_class}' and @id='{input_id}' and @name='{input_name}' and @aria-label='{aria_label}']"

    # Construct the XPath manually using the tag name and attributes
    return xpath

In [ ]:
def next_element(tabs,driver,url,all_elements,element,index):
    new_tab_handle = driver.window_handles[index]
    driver.switch_to.window(new_tab_handle)
    if element["type"]=="button":
        try:
            url=element["element"].get_attribute('href')
            open_tab(url,tabs,element,driver)
        except ElementNotInteractableException:
                all_elements.remove(element)
        except StaleElementReferenceException:
            print("Button : StaleElementReferenceException")
    else:
        if element["type"]=="input":
            try:
                xpath = input_xpath(element["element"])
                script = f"window.open('{driver.current_url}', '_blank');"
                driver.execute_script(script)
                # Switch to the new tab
                driver.switch_to.window(driver.window_handles[-1])
                text_to_write = "Hello"
                old_element = element
                element = driver.find_element(By.XPATH, xpath)
                element.send_keys(text_to_write)
                url= driver.current_url
                length = len(tabs)
                if length!=get_page(tabs,url,driver):
                    driver.close()
            except ElementNotInteractableException:
                all_elements.remove(old_element) 
            except StaleElementReferenceException:
                print("Input : StaleElementReferenceException")
                
            
            

In [ ]:
def script(url):
    # Set up Chrome options
    chrome_options = Options()
    service = Service(executable_path='C:\Program Files (x86)\chromedriver.exe')
    chrome_options.binary_location = 'C:\Program Files\Google\Chrome\Application\chrome.exe'  # Specify the path to the Chrome browser executable if needed
    chrome_options.add_argument('--headless')  # Run Chrome in headless mode
    chrome_options.add_argument('--page-load-timeout=60')

    # Set up Chrome WebDriver with the local ChromeDriver executable
    driver = webdriver.Chrome(options=chrome_options, service=service)
    length = 0
    tabs = [{"page": url,"index":0,"page_elements" : []}]
    
    while length<len(tabs):
        start_time = time.time()
        driver.switch_to.window(driver.window_handles[length])
        if length ==0:
            current_url = url
        else:
            current_url = driver.current_url
        all_elements = get_page_elements(driver,current_url)
        for element in all_elements:
            next_element(tabs,driver,current_url,all_elements,element,length)
        tabs[length]["page_elements"] = all_elements
        end_time = time.time()
        execution_time = end_time - start_time
        print("------------------------")
        print(current_url)
        print(execution_time)
        print("------------------------")
        length += 1
    return tabs
        
    
        
    
    
    
        
    

In [ ]:
tabs = script("https://www.ip-label.fr/")

In [ ]:
for page in tabs:
    if page["index"]==1:
        driver.switch_to.window(driver.window_handles[1])
        page
        


        


In [3]:
def scriptt(url):
    # Set up Chrome options
    chrome_options = Options()
    service = Service(executable_path='C:\Program Files (x86)\chromedriver.exe')
    chrome_options.binary_location = 'C:\Program Files\Google\Chrome\Application\chrome.exe'  # Specify the path to the Chrome browser executable if needed
    #chrome_options.add_argument('--headless')  # Run Chrome in headless mode
    print(url)
    # Set up Chrome WebDriver with the local ChromeDriver executable
    driver = webdriver.Chrome(options=chrome_options, service=service)
    driver.get(url)
    element = {'type': 'button', 'next': 'https://www.ip-label.fr/formulaire-contact-supervision-des-applications-metier', 'xPath': '/html[1]/body[1]/div[2]/div[1]/main[1]/div[1]/div[1]/div[1]/article[1]/div[1]/div[1]/section[1]/div[1]/div[1]/div[1]/div[3]/div[1]/div[1]/a[1]'}
    elem = driver.find_element(By.XPATH, element["xPath"])
    
    
    
scriptt('https://www.ip-label.fr/choix-action')

https://www.ip-label.fr/choix-action


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html[1]/body[1]/div[2]/div[1]/main[1]/div[1]/div[1]/div[1]/article[1]/div[1]/div[1]/section[1]/div[1]/div[1]/div[1]/div[3]/div[1]/div[1]/a[1]"}
  (Session info: chrome=116.0.5845.111)
Stacktrace:
	GetHandleVerifier [0x00007FF759BB52A2+57122]
	(No symbol) [0x00007FF759B2EA92]
	(No symbol) [0x00007FF7599FE3AB]
	(No symbol) [0x00007FF759A37D3E]
	(No symbol) [0x00007FF759A37E2C]
	(No symbol) [0x00007FF759A70B67]
	(No symbol) [0x00007FF759A5701F]
	(No symbol) [0x00007FF759A6EB82]
	(No symbol) [0x00007FF759A56DB3]
	(No symbol) [0x00007FF759A2D2B1]
	(No symbol) [0x00007FF759A2E494]
	GetHandleVerifier [0x00007FF759E5EF82+2849794]
	GetHandleVerifier [0x00007FF759EB1D24+3189156]
	GetHandleVerifier [0x00007FF759EAACAF+3160367]
	GetHandleVerifier [0x00007FF759C46D06+653702]
	(No symbol) [0x00007FF759B3A208]
	(No symbol) [0x00007FF759B362C4]
	(No symbol) [0x00007FF759B363F6]
	(No symbol) [0x00